# Semantic Search over crosswords data by Fasttext embeddings

In [1]:
import fasttext
import numpy as np
import sys
import math
from typing import List
import re
from pprint import pprint as print
from tqdm import tqdm
import random

Using [fasttext Persian embeddings model binary](https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz) which was trained using CBOW on Persian texts. The model consists of embeddings for 2M words.

In [2]:
farsi = fasttext.load_model("model/cc.fa.300.bin")

In [3]:
cw_lines = []

In [4]:
data_files = ["cw.train.tsv", "cw.dev.tsv", "cw.test.tsv"]
index = 0
for file in data_files:
    for i, line in enumerate(open(file, encoding="utf-8")):
        cw_lines.append([index, line.strip()])
        index += 1

In [6]:
cw_lines[0], len(cw_lines), cw_lines[-1]

([0, 'بی حال و سست\tکسل'], 30157, [30156, 'آرام و یواش\tاهسته'])

In [7]:
def embed(word: str):
    return farsi.get_word_vector(word)

In [8]:
def cosine(vec_1: List[float], vec_2: List[float]):
    #return np.dot(vec_1, vec_2)
    return float(np.dot(vec_1, vec_2)/(np.linalg.norm(vec_1) * np.linalg.norm(vec_2)))

def sentence_similarity(evd_emb, sentence_embs):
    similarities = [cosine(evd_emb, sentence_emb[1]) for sentence_emb in sentence_embs]
    return similarities

Make embeddings of a sentence based on each word embeddings

In [9]:
def embed_sentence(sentence: str, window: int = sys.maxsize):
    tokens = sentence.split()
    vectors = [embed(word) for word in tokens]

    averages = []
    averages.append(np.average(np.array(vectors), axis=0))
    return averages[0]

Embed all dataset records

In [10]:
all_vs_embed = []
vocabs = []
for i, entry in cw_lines:
    sentence_embeddings = embed_sentence(entry)
    all_vs_embed.append([i, sentence_embeddings])
    vocabs.append([i, entry])

Calculate similarity of a sentence embeddings agains all dataset records embeddings. Then return the sorted list.

In [11]:
def qa_sim(q_embed, i=-1):
    all_embeds = all_vs_embed
    altered_vocabs = vocabs

    if i != -1:
        all_embeds = [x for x in all_vs_embed if x[0] != i]
        altered_vocabs = [x for x in vocabs if x[0] != i]

    return sorted(list(zip(sentence_similarity(q_embed, all_embeds), altered_vocabs)), reverse=True)

Testing sample queries

In [12]:
%%time

sentences = ["اسیر و گرفتار"
, "داستانی به قلم آندره ژید فرانسوی"
, "جهل"
, "حیوان"
, "فرمانی نظامی"
, "پیام آسمانی"
, "نهفتگی"
, "افزونی برنج پس از پخته شدن"
, "گاز نجیب سبک"
, "خوش طعم"
, "شاعر یوش"
, "آندره ژید"]

for sentence in sentences:
    query_embeddings = embed_sentence(sentence)
    print(sentence)
    print(qa_sim(query_embeddings)[:5])

'اسیر و گرفتار'
[(0.9431297779083252, [17890, 'دربند و گرفتار\tاسیر']),
 (0.9431297779083252, [15662, 'گرفتار و اسیر\tدربند']),
 (0.8334949016571045, [26791, 'اسارت و گرفتاری\tاسیری']),
 (0.8309114575386047, [16067, 'گرفتار و مشغول\tدرگیر']),
 (0.783309280872345, [6320, 'گرفتار و مبتلا\tدچار'])]
'داستانی به قلم آندره ژید فرانسوی'
[(0.72873455286026, [12301, 'کتابی به قلم ناهید طباطبایی\tچهلسالگی']),
 (0.6134222149848938,
  [23343, 'کتابی از سهراب سپهری که به نثر نگاشته شده\u200cاست\tاتاقابی']),
 (0.60152667760849, [13677, 'اثري از آندره مالرو\tروزهایخشم']),
 (0.5866066217422485,
  [17068, 'فیلمی به کارگردانی و نویسندگی محمدعلی سجادی\tرنگشب']),
 (0.5800595283508301, [16206, 'سرهنگ به فرانسوی\tکلنل'])]
'جهل'
[(0.9682827591896057, [1237, 'نادانی\tجهل']),
 (0.7587428689002991, [27300, 'جهالت\tنادانی']),
 (0.7587428689002991, [9245, 'نادانی\tجهالت']),
 (0.6205307841300964, [26077, 'نادانی و حماقت\tسفاهت']),
 (0.5980551242828369, [13951, '38\tجهل'])]
'حیوان'
[(0.8916774988174438, [22791, 'جا

We calculate similarity of each record of dataset against all other records of the dataset.

In [14]:
%%time

firsts = []
top_fives= []
#for i in tqdm(range(len(devset))):
for i in tqdm(range(len(cw_lines))):
    q = cw_lines[i][1].split("\t")[0]
    a = cw_lines[i][1].split("\t")[1]
    inx = cw_lines[i][0]

    query_embeddings = embed_sentence(q)
    res = qa_sim(query_embeddings)[0:6] #, inx)[0:6]

    res = [x for x in res if x[1][0] != inx]
    if a in res[0][1][1]:
        firsts.append({'q': q, 'a': a, 'found': res[0][1][1]})
        
    is_in_top_fives = [x[1] for x in res if a in x[1][1]]
    if is_in_top_fives:
        top_five = {
            'q': q,
            'a': a,
            'found': []
        }
        for five in is_in_top_fives:
            top_five['found'].append(five)

        top_fives.append(top_five)

100%|██████████| 30157/30157 [3:02:06<00:00,  2.76it/s]  

CPU times: user 3h 2min 17s, sys: 20 s, total: 3h 2min 37s
Wall time: 3h 2min 6s


precision percent of first highest score and five highest score

In [43]:
round(len(firsts)/len(cw_lines)*100, 2), round(len(top_fives)/len(cw_lines)*100, 2)

(13.21, 26.34)

In [36]:
firsts

[{'q': 'دستگاه کنترل از راه دور',
  'a': 'ریموت',
  'found': 'دستگاه هدايت از دور\tریموتکنترل'},
 {'q': 'فحش', 'a': 'ناسزا', 'found': 'فحش و دشنام\tناسزا'},
 {'q': 'ice', 'a': 'یخ', 'found': 'یخ فروش\tیخی'},
 {'q': 'نیم صدای ساعت', 'a': 'تیک', 'found': 'صدای ساعت\tتیکتاک'},
 {'q': 'نام', 'a': 'اسم', 'found': 'اسم\tنام'},
 {'q': 'ریاکاری و نفاق', 'a': 'دورویی', 'found': 'نفاق و ریاکاری\tدورویی'},
 {'q': 'مال و ثروت', 'a': 'دارایی', 'found': 'ثروت و دارایی\tمال'},
 {'q': 'دریوزه گری', 'a': 'گد', 'found': 'دریوزه گری\tگدایی'},
 {'q': 'همگی و کلاً', 'a': 'یکجا', 'found': 'همگی و کلا\tیکجا'},
 {'q': 'یکی از مزه ها', 'a': 'تلخ', 'found': 'از مزه ها\tتلخ'},
 {'q': 'عدد بلبل', 'a': 'هزار', 'found': 'بلبل\tهزار'},
 {'q': 'مامور', 'a': 'گماشته', 'found': 'گماشته\tمامور'}]

In [17]:
random.choice(firsts)

{'q': 'از ورزش\u200cهای گروهی پرطرفدار',
 'a': 'فوتبال',
 'found': 'از ورزش\u200cهای گروهی\tفوتبال'}

In [18]:
refined_firsts = [x for x in firsts if x['a'] == x['found'].split("\t")[1]]

Precision of exact same match for the first highest score

In [44]:
round(len(refined_firsts)/len(cw_lines)*100, 2)

6.27

In [48]:
random.choice(refined_firsts)

{'q': 'قیمتی و گران مایه',
 'a': 'بهاگیر',
 'found': 'هر چیز قیمتی و گران مایه\tبهاگیر'}

Precision of exact same match in the top five highest scores

In [35]:
refined_top_fives = []
for five in top_fives:
    for f in five['found']:
        if five['a'] == f[1].split("\t")[1]:
            temp = {
                'q': five['q'],
                'a': five['a'],
                'found': f[1]
            }
            refined_top_fives.append(temp)

In [46]:
round(len(refined_top_fives)/len(cw_lines)*100, 2)

18.27

In [40]:
random.choice(refined_top_fives)

{'q': 'سردی و خنکی', 'a': 'برودت', 'found': 'سردی\tبرودت'}